# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Dec 28 2022 11:13AM,253915,10,SO94483,"Senseonics, Incorporated",Released
1,Dec 28 2022 11:10AM,253914,10,MSP217044,"Methapharm, Inc.",Released
2,Dec 28 2022 11:10AM,253914,10,MSP217045,"Methapharm, Inc.",Released
3,Dec 28 2022 11:10AM,253913,10,0086271365,ISDIN Corporation,Released
4,Dec 28 2022 11:10AM,253913,10,0086271366,ISDIN Corporation,Released
5,Dec 28 2022 11:10AM,253913,10,0086271364,ISDIN Corporation,Released
6,Dec 28 2022 11:04AM,253912,10,MSP217043,"Methapharm, Inc.",Released
7,Dec 28 2022 11:04AM,253912,10,MSP217048,"Methapharm, Inc.",Released
8,Dec 28 2022 11:04AM,253912,10,MSP217049,"Methapharm, Inc.",Released
9,Dec 28 2022 10:59AM,253911,19,ACG-2102492254,ACG North America LLC,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
38,253911,Released,1
39,253912,Released,3
40,253913,Released,3
41,253914,Released,2
42,253915,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
253911,NaN,1.0
253912,NaN,3.0
253913,NaN,3.0
253914,NaN,2.0
253915,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
253829,0.0,3.0
253830,0.0,49.0
253832,0.0,22.0
253850,0.0,1.0
253852,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
253829,0,3
253830,0,49
253832,0,22
253850,0,1
253852,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,253829,0,3
1,253830,0,49
2,253832,0,22
3,253850,0,1
4,253852,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,253829,,3
1,253830,,49
2,253832,,22
3,253850,,1
4,253852,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Dec 28 2022 11:13AM,253915,10,"Senseonics, Incorporated"
1,Dec 28 2022 11:10AM,253914,10,"Methapharm, Inc."
3,Dec 28 2022 11:10AM,253913,10,ISDIN Corporation
6,Dec 28 2022 11:04AM,253912,10,"Methapharm, Inc."
9,Dec 28 2022 10:59AM,253911,19,ACG North America LLC
10,Dec 28 2022 10:58AM,253905,10,Eywa Pharma Inc.
21,Dec 28 2022 10:57AM,253910,19,ACG North America LLC
22,Dec 28 2022 10:56AM,253909,10,ISDIN Corporation
25,Dec 28 2022 10:55AM,253908,10,"Snap Medical Industries, LLC"
26,Dec 28 2022 10:51AM,253907,19,ACG North America LLC


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Dec 28 2022 11:13AM,253915,10,"Senseonics, Incorporated",,1
1,Dec 28 2022 11:10AM,253914,10,"Methapharm, Inc.",,2
2,Dec 28 2022 11:10AM,253913,10,ISDIN Corporation,,3
3,Dec 28 2022 11:04AM,253912,10,"Methapharm, Inc.",,3
4,Dec 28 2022 10:59AM,253911,19,ACG North America LLC,,1
5,Dec 28 2022 10:58AM,253905,10,Eywa Pharma Inc.,,11
6,Dec 28 2022 10:57AM,253910,19,ACG North America LLC,,1
7,Dec 28 2022 10:56AM,253909,10,ISDIN Corporation,,3
8,Dec 28 2022 10:55AM,253908,10,"Snap Medical Industries, LLC",,1
9,Dec 28 2022 10:51AM,253907,19,ACG North America LLC,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Dec 28 2022 11:13AM,253915,10,"Senseonics, Incorporated",1,
1,Dec 28 2022 11:10AM,253914,10,"Methapharm, Inc.",2,
2,Dec 28 2022 11:10AM,253913,10,ISDIN Corporation,3,
3,Dec 28 2022 11:04AM,253912,10,"Methapharm, Inc.",3,
4,Dec 28 2022 10:59AM,253911,19,ACG North America LLC,1,
5,Dec 28 2022 10:58AM,253905,10,Eywa Pharma Inc.,11,
6,Dec 28 2022 10:57AM,253910,19,ACG North America LLC,1,
7,Dec 28 2022 10:56AM,253909,10,ISDIN Corporation,3,
8,Dec 28 2022 10:55AM,253908,10,"Snap Medical Industries, LLC",1,
9,Dec 28 2022 10:51AM,253907,19,ACG North America LLC,1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Dec 28 2022 11:13AM,253915,10,"Senseonics, Incorporated",1,
1,Dec 28 2022 11:10AM,253914,10,"Methapharm, Inc.",2,
2,Dec 28 2022 11:10AM,253913,10,ISDIN Corporation,3,
3,Dec 28 2022 11:04AM,253912,10,"Methapharm, Inc.",3,
4,Dec 28 2022 10:59AM,253911,19,ACG North America LLC,1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Dec 28 2022 11:13AM,253915,10,"Senseonics, Incorporated",1.0,NaN
1,Dec 28 2022 11:10AM,253914,10,"Methapharm, Inc.",2.0,NaN
2,Dec 28 2022 11:10AM,253913,10,ISDIN Corporation,3.0,NaN
3,Dec 28 2022 11:04AM,253912,10,"Methapharm, Inc.",3.0,NaN
4,Dec 28 2022 10:59AM,253911,19,ACG North America LLC,1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Dec 28 2022 11:13AM,253915,10,"Senseonics, Incorporated",1.0,0.0
1,Dec 28 2022 11:10AM,253914,10,"Methapharm, Inc.",2.0,0.0
2,Dec 28 2022 11:10AM,253913,10,ISDIN Corporation,3.0,0.0
3,Dec 28 2022 11:04AM,253912,10,"Methapharm, Inc.",3.0,0.0
4,Dec 28 2022 10:59AM,253911,19,ACG North America LLC,1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,7362582,328.0,0.0
12,253892,0.0,1.0
19,3046655,41.0,9.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,7362582,328.0,0.0
1,12,253892,0.0,1.0
2,19,3046655,41.0,9.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,328.0,0.0
1,12,0.0,1.0
2,19,41.0,9.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,328.0
1,12,Released,0.0
2,19,Released,41.0
3,10,Executing,0.0
4,12,Executing,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,19
Status,,,
Executing,0.0,1.0,9.0
Released,328.0,0.0,41.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,19
0,Executing,0.0,1.0,9.0
1,Released,328.0,0.0,41.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,19
0,Executing,0.0,1.0,9.0
1,Released,328.0,0.0,41.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()